# Homework and bake-off: Relation extraction using distant supervision

In [1]:
__author__ = "Bill MacCartney and Christopher Potts"
__version__ = "CS224u, Stanford, Fall 2020"

## Contents

1. [Overview](#Overview)
1. [Set-up](#Set-up)
1. [Baselines](#Baselines)
  1. [Hand-build feature functions](#Hand-build-feature-functions)
  1. [Distributed representations](#Distributed-representations)
1. [Homework questions](#Homework-questions)
  1. [Different model factory [1 points]](#Different-model-factory-[1-points])
  1. [Directional unigram features [1.5 points]](#Directional-unigram-features-[1.5-points])
  1. [The part-of-speech tags of the "middle" words [1.5 points]](#The-part-of-speech-tags-of-the-"middle"-words-[1.5-points])
  1. [Bag of Synsets [2 points]](#Bag-of-Synsets-[2-points])
  1. [Your original system [3 points]](#Your-original-system-[3-points])
1. [Bake-off [1 point]](#Bake-off-[1-point])

## Overview

This homework and associated bake-off are devoted to developing really effective relation extraction systems using distant supervision. 

As with the previous assignments, this notebook first establishes a baseline system. The initial homework questions ask you to create additional baselines and suggest areas for innovation, and the final homework question asks you to develop an original system for you to enter into the bake-off.

## Set-up

See [the first notebook in this unit](rel_ext_01_task.ipynb#Set-up) for set-up instructions.

In [2]:
import numpy as np
import os
import rel_ext
from sklearn.linear_model import LogisticRegression
import utils

As usual, we unite our corpus and KB into a dataset, and create some splits for experimentation:

In [10]:
data_path = '/home/datawrestler/data/stanford-nlu/data'

if 'IS_GRADESCOPE_ENV' in os.environ:
    data_path = 'data'

rel_ext_data_home = os.path.join(data_path, 'rel_ext_data')

In [12]:
corpus = rel_ext.Corpus(os.path.join(rel_ext_data_home, 'corpus.tsv.gz'))

In [13]:
kb = rel_ext.KB(os.path.join(rel_ext_data_home, 'kb.tsv.gz'))

In [14]:
dataset = rel_ext.Dataset(corpus, kb)

You are not wedded to this set-up for splits. The bake-off will be conducted on a previously unseen test-set, so all of the data in `dataset` is fair game:

In [15]:
splits = dataset.build_splits(
    split_names=['tiny', 'train', 'dev'],
    split_fracs=[0.01, 0.79, 0.20],
    seed=1)

## Baselines

### Hand-build feature functions

In [8]:
def simple_bag_of_words_featurizer(kbt, corpus, feature_counter):
    for ex in corpus.get_examples_for_entities(kbt.sbj, kbt.obj):
        for word in ex.middle.split(' '):
            feature_counter[word] += 1
    for ex in corpus.get_examples_for_entities(kbt.obj, kbt.sbj):
        for word in ex.middle.split(' '):
            feature_counter[word] += 1
    return feature_counter

In [9]:
featurizers = [simple_bag_of_words_featurizer]

In [10]:
model_factory = lambda: LogisticRegression(fit_intercept=True, solver='liblinear')

In [11]:
baseline_results = rel_ext.experiment(
    splits,
    train_split='train',
    test_split='dev',
    featurizers=featurizers,
    model_factory=model_factory,
    verbose=True)

relation              precision     recall    f-score    support       size
------------------    ---------  ---------  ---------  ---------  ---------
adjoins                   0.884      0.382      0.700        340       5716
author                    0.791      0.558      0.730        509       5885
capital                   0.579      0.232      0.445         95       5471
contains                  0.800      0.590      0.747       3904       9280
film_performance          0.782      0.563      0.726        766       6142
founders                  0.799      0.397      0.665        380       5756
genre                     0.587      0.159      0.381        170       5546
has_sibling               0.851      0.240      0.564        499       5875
has_spouse                0.892      0.320      0.657        594       5970
is_a                      0.677      0.227      0.485        497       5873
nationality               0.570      0.163      0.380        301       5677
parents     

Studying model weights might yield insights:

In [12]:
rel_ext.examine_model_weights(baseline_results)

Highest and lowest feature weights for relation adjoins:

     2.556 Córdoba
     2.514 Taluks
     2.461 Province
     ..... .....
    -1.470 who
    -1.480 Cook
    -1.489 he

Highest and lowest feature weights for relation author:

     2.741 books
     2.676 book
     2.424 author
     ..... .....
    -2.722 Alice
    -2.994 Daisy
    -6.985 1865

Highest and lowest feature weights for relation capital:

     3.798 capital
     1.824 km
     1.786 now
     ..... .....
    -1.282 largest
    -1.339 South
    -1.985 Madras

Highest and lowest feature weights for relation contains:

     2.789 third-largest
     2.620 bordered
     2.153 towns
     ..... .....
    -2.383 who
    -2.486 Madras
    -6.046 Bronx

Highest and lowest feature weights for relation film_performance:

     3.986 starring
     3.538 alongside
     3.528 movie
     ..... .....
    -1.883 Keystone
    -1.993 Zigzag
    -3.990 double

Highest and lowest feature weights for relation founders:

     4.584 founder
  

### Distributed representations

This simple baseline sums the GloVe vector representations for all of the words in the "middle" span and feeds those representations into the standard `LogisticRegression`-based `model_factory`. The crucial parameter that enables this is `vectorize=False`. This essentially says to `rel_ext.experiment` that your featurizer or your model will do the work of turning examples into vectors; in that case, `rel_ext.experiment` just organizes these representations by relation type.

In [13]:
GLOVE_HOME = os.path.join(data_path, 'glove.6B')

glove_lookup = utils.glove2dict(
    os.path.join(GLOVE_HOME, 'glove.6B.300d.txt'))

In [15]:
def glove_middle_featurizer(kbt, corpus, np_func=np.sum):
    reps = []
    for ex in corpus.get_examples_for_entities(kbt.sbj, kbt.obj):
        for word in ex.middle.split():
            rep = glove_lookup.get(word)
            if rep is not None:
                reps.append(rep)
    # A random representation of the right dimensionality if the
    # example happens not to overlap with GloVe's vocabulary:
    if len(reps) == 0:
        dim = len(next(iter(glove_lookup.values())))
        return utils.randvec(n=dim)
    else:
        return np_func(reps, axis=0)

In [16]:
glove_results = rel_ext.experiment(
    splits,
    train_split='train',
    test_split='dev',
    featurizers=[glove_middle_featurizer],
    vectorize=False, # Crucial for this featurizer!
    verbose=True)

relation              precision     recall    f-score    support       size
------------------    ---------  ---------  ---------  ---------  ---------
adjoins                   0.851      0.453      0.724        340       5716
author                    0.882      0.424      0.725        509       5885
capital                   0.564      0.232      0.438         95       5471
contains                  0.662      0.400      0.585       3904       9280
film_performance          0.789      0.317      0.608        766       6142
founders                  0.771      0.239      0.534        380       5756
genre                     0.474      0.053      0.183        170       5546
has_sibling               0.756      0.248      0.537        499       5875
has_spouse                0.866      0.360      0.676        594       5970
is_a                      0.730      0.147      0.407        497       5873
nationality               0.615      0.213      0.446        301       5677
parents     

With the same basic code design, one can also use the PyTorch models included in the course repo, or write new ones that are better aligned with the task. For those models, it's likely that the featurizer will just return a list of tokens (or perhaps a list of lists of tokens), and the model will map those into vectors using an embedding.

## Homework questions

Please embed your homework responses in this notebook, and do not delete any cells from the notebook. (You are free to add as many cells as you like as part of your responses.)

### Different model factory [1 points]

The code in `rel_ext` makes it very easy to experiment with other classifier models: one need only redefine the `model_factory` argument. This question asks you to assess a [Support Vector Classifier](https://scikit-learn.org/stable/modules/generated/sklearn.svm.SVC.html).

__To submit:__ A wrapper function `run_svm_model_factory` that does the following: 

1. Uses `rel_ext.experiment` with the model factory set to one based in an `SVC` with `kernel='linear'` and all other arguments left with default values. 
1. Trains on the 'train' part of `splits`.
1. Assesses on the `dev` part of `splits`.
1. Uses `featurizers` as defined above. 
1. Returns the return value of `rel_ext.experiment` for this set-up.

The function `test_run_svm_model_factory` will check that your function conforms to these general specifications.

In [20]:
def run_svm_model_factory():

    ##### YOUR CODE HERE
    from sklearn.svm import SVC
    
    model = SVC(
        kernel='linear',
    )
    
    svc_results = rel_ext.experiment(
        splits,
        train_split='train',
        test_split='dev',
        model_factory=(lambda: model),
        featurizers=[glove_middle_featurizer],
        vectorize=False, # Crucial for this featurizer!
        verbose=True
    )
    
    return svc_results

In [21]:
def test_run_svm_model_factory(run_svm_model_factory):
    results = run_svm_model_factory()
    assert 'featurizers' in results, \
        "The return value of `run_svm_model_factory` seems not to be correct"
    # Check one of the models to make sure it's an SVC:
    assert 'SVC' in results['models']['adjoins'].__class__.__name__, \
        "It looks like the model factor wasn't set to use an SVC."

In [22]:
if 'IS_GRADESCOPE_ENV' not in os.environ:
    test_run_svm_model_factory(run_svm_model_factory)

relation              precision     recall    f-score    support       size
------------------    ---------  ---------  ---------  ---------  ---------
adjoins                   0.192      0.056      0.129        340       5716
author                    0.000      0.000      0.000        509       5885
capital                   0.070      0.063      0.068         95       5471
contains                  0.765      0.069      0.254       3904       9280
film_performance          0.186      0.025      0.081        766       6142
founders                  0.163      0.042      0.104        380       5756
genre                     0.000      0.000      0.000        170       5546
has_sibling               0.111      0.020      0.058        499       5875
has_spouse                0.200      0.034      0.101        594       5970
is_a                      0.069      0.012      0.036        497       5873
nationality               0.162      0.053      0.115        301       5677
parents     

### Directional unigram features [1.5 points]

The current bag-of-words representation makes no distinction between "forward" and "reverse" examples. But, intuitively, there is big difference between _X and his son Y_ and _Y and his son X_. This question asks you to modify `simple_bag_of_words_featurizer` to capture these differences. 

__To submit:__

1. A feature function `directional_bag_of_words_featurizer` that is just like `simple_bag_of_words_featurizer` except that it distinguishes "forward" and "reverse". To do this, you just need to mark each word feature for whether it is derived from a subject–object example or from an object–subject example.  The included function `test_directional_bag_of_words_featurizer` should help verify that you've done this correctly.

2. A call to `rel_ext.experiment` with `directional_bag_of_words_featurizer` as the only featurizer. (Aside from this, use all the default values for `rel_ext.experiment` as exemplified above in this notebook.)

3. `rel_ext.experiment` returns some of the core objects used in the experiment. How many feature names does the `vectorizer` have for the experiment run in the previous step? Include the code needed for getting this value. (Note: we're partly asking you to figure out how to get this value by using the sklearn documentation, so please don't ask how to do it!)

In [54]:
def directional_bag_of_words_featurizer(kbt, corpus, feature_counter):
    # Append these to the end of the keys you add/access in
    # `feature_counter` to distinguish the two orders. You'll
    # need to use exactly these strings in order to pass
    # `test_directional_bag_of_words_featurizer`.
    subject_object_suffix = "_SO"
    object_subject_suffix = "_OS"

    ##### YOUR CODE HERE
    for ex in corpus.get_examples_for_entities(kbt.sbj, kbt.obj):
        for word in ex.middle.split('  '):
            feature_counter[f"{word}{subject_object_suffix}"] += 1
        
    for ex in corpus.get_examples_for_entities(kbt.obj, kbt.sbj):
        for word in ex.middle.split(' '):
            feature_counter[f"{word}{object_subject_suffix}"] += 1
            
    return feature_counter

# Call to `rel_ext.experiment`:
##### YOUR CODE HERE

model_factory = lambda: LogisticRegression(fit_intercept=True, solver='liblinear')

featurizers = [directional_bag_of_words_featurizer]

baseline_results = rel_ext.experiment(
    splits,
    train_split='train',
    test_split='dev',
    featurizers=featurizers,
    model_factory=model_factory,
    verbose=True)

relation              precision     recall    f-score    support       size
------------------    ---------  ---------  ---------  ---------  ---------
adjoins                   0.917      0.291      0.641        340       5716
author                    0.871      0.519      0.767        509       5885
capital                   0.481      0.137      0.320         95       5471
contains                  0.854      0.506      0.751       3904       9280
film_performance          0.835      0.510      0.741        766       6142
founders                  0.809      0.289      0.595        380       5756
genre                     0.759      0.241      0.531        170       5546
has_sibling               0.794      0.108      0.350        499       5875
has_spouse                0.869      0.190      0.507        594       5970
is_a                      0.679      0.111      0.335        497       5873
nationality               0.691      0.156      0.410        301       5677
parents     

In [55]:
def test_directional_bag_of_words_featurizer(corpus):
    from collections import defaultdict
    kbt = rel_ext.KBTriple(rel='worked_at', sbj='Randall_Munroe', obj='xkcd')
    feature_counter = defaultdict(int)
    # Make sure `feature_counter` is being updated, not reinitialized:
    feature_counter['is_OS'] += 5
    feature_counter = directional_bag_of_words_featurizer(kbt, corpus, feature_counter)
    expected = defaultdict(
        int, {'is_OS':6,'a_OS':1,'webcomic_OS':1,'created_OS':1,'by_OS':1})
    assert feature_counter == expected, \
        "Expected:\n{}\nGot:\n{}".format(expected, feature_counter)

In [56]:
if 'IS_GRADESCOPE_ENV' not in os.environ:
    test_directional_bag_of_words_featurizer(corpus)

### The part-of-speech tags of the "middle" words [1.5 points]

Our corpus distribution contains part-of-speech (POS) tagged versions of the core text spans. Let's begin to explore whether there is information in these sequences, focusing on `middle_POS`.

__To submit:__

1. A feature function `middle_bigram_pos_tag_featurizer` that is just like `simple_bag_of_words_featurizer` except that it creates a feature for bigram POS sequences. For example, given 

  `The/DT dog/N napped/V`
  
   we obtain the list of bigram POS sequences
  
   `b = ['<s> DT', 'DT N', 'N V', 'V </s>']`. 
   
   Of course, `middle_bigram_pos_tag_featurizer` should return count dictionaries defined in terms of such bigram POS lists, on the model of `simple_bag_of_words_featurizer`.  Don't forget the start and end tags, to model those environments properly! The included function `test_middle_bigram_pos_tag_featurizer` should help verify that you've done this correctly.

2. A call to `rel_ext.experiment` with `middle_bigram_pos_tag_featurizer` as the only featurizer. (Aside from this, use all the default values for `rel_ext.experiment` as exemplified above in this notebook.)

In [85]:
def middle_bigram_pos_tag_featurizer(kbt, corpus, feature_counter):

    ##### YOUR CODE HERE
    for ex in corpus.get_examples_for_entities(kbt.sbj, kbt.obj):
        bigrams = get_tag_bigrams(ex.middle_POS)
        for bigram in bigrams:
            bigram = ' '.join(bigram)
            feature_counter[bigram] += 1
        
    for ex in corpus.get_examples_for_entities(kbt.obj, kbt.sbj):
        bigrams = get_tag_bigrams(ex.middle_POS)
        for bigram in bigrams:
            bigram = ' '.join(bigram)
            feature_counter[bigram] += 1

    return feature_counter


def get_tag_bigrams(s):
    """Suggested helper method for `middle_bigram_pos_tag_featurizer`.
    This should be defined so that it returns a list of str, where each
    element is a POS bigram."""
    # The values of `start_symbol` and `end_symbol` are defined
    # here so that you can use `test_middle_bigram_pos_tag_featurizer`.
    start_symbol = "<s>"
    end_symbol = "</s>"

    ##### YOUR CODE HERE
    tags = get_tags(s)
    tags = [start_symbol] + tags + [end_symbol]
    bigrams = list(zip(tags[:-1], tags[1:]))
    return bigrams
    
def get_tags(s):
    """Given a sequence of word/POS elements (lemmas), this function
    returns a list containing just the POS elements, in order.
    """
    return [parse_lem(lem)[1] for lem in s.strip().split(' ') if lem]


def parse_lem(lem):
    """Helper method for parsing word/POS elements. It just splits
    on the rightmost / and returns (word, POS) as a tuple of str."""
    return lem.strip().rsplit('/', 1)

# Call to `rel_ext.experiment`:
##### YOUR CODE HERE
model_factory = lambda: LogisticRegression(fit_intercept=True, solver='liblinear')

featurizers = [middle_bigram_pos_tag_featurizer]

baseline_results = rel_ext.experiment(
    splits,
    train_split='train',
    test_split='dev',
    featurizers=featurizers,
    model_factory=model_factory,
    verbose=True)

relation              precision     recall    f-score    support       size
------------------    ---------  ---------  ---------  ---------  ---------
adjoins                   0.856      0.332      0.651        340       5716
author                    0.736      0.301      0.570        509       5885
capital                   0.483      0.147      0.332         95       5471
contains                  0.751      0.598      0.714       3904       9280
film_performance          0.663      0.432      0.599        766       6142
founders                  0.571      0.168      0.386        380       5756
genre                     0.611      0.194      0.427        170       5546
has_sibling               0.702      0.160      0.419        499       5875
has_spouse                0.802      0.266      0.572        594       5970
is_a                      0.600      0.181      0.410        497       5873
nationality               0.350      0.070      0.194        301       5677
parents     

In [86]:
def test_middle_bigram_pos_tag_featurizer(corpus):
    from collections import defaultdict
    kbt = rel_ext.KBTriple(rel='worked_at', sbj='Randall_Munroe', obj='xkcd')
    feature_counter = defaultdict(int)
    # Make sure `feature_counter` is being updated, not reinitialized:
    feature_counter['<s> VBZ'] += 5
    feature_counter = middle_bigram_pos_tag_featurizer(kbt, corpus, feature_counter)
    expected = defaultdict(
        int, {'<s> VBZ':6,'VBZ DT':1,'DT JJ':1,'JJ VBN':1,'VBN IN':1,'IN </s>':1})
    assert feature_counter == expected, \
        "Expected:\n{}\nGot:\n{}".format(expected, feature_counter)

In [87]:
if 'IS_GRADESCOPE_ENV' not in os.environ:
    test_middle_bigram_pos_tag_featurizer(corpus)

### Bag of Synsets [2 points]

The following allows you to use NLTK's WordNet API to get the synsets compatible with _dog_ as used as a noun:

```
from nltk.corpus import wordnet as wn
dog = wn.synsets('dog', pos='n')
dog
[Synset('dog.n.01'),
 Synset('frump.n.01'),
 Synset('dog.n.03'),
 Synset('cad.n.01'),
 Synset('frank.n.02'),
 Synset('pawl.n.01'),
 Synset('andiron.n.01')]
```

This question asks you to create synset-based features from the word/tag pairs in `middle_POS`.

__To submit:__

1. A feature function `synset_featurizer` that is just like `simple_bag_of_words_featurizer` except that it returns a list of synsets derived from `middle_POS`. Stringify these objects with `str` so that they can be `dict` keys. Use `convert_tag` (included below) to convert tags to `pos` arguments usable by `wn.synsets`. The included function `test_synset_featurizer` should help verify that you've done this correctly.

2. A call to `rel_ext.experiment` with `synset_featurizer` as the only featurizer. (Aside from this, use all the default values for `rel_ext.experiment`.)

In [120]:
import nltk
from nltk.corpus import wordnet as wn
try:
    synsets = wn.synsets('dog', pos='n')
except:
    nltk.download('wordnet')

def synset_featurizer(kbt, corpus, feature_counter):

    ##### YOUR CODE HERE
    for ex in corpus.get_examples_for_entities(kbt.sbj, kbt.obj):
        s = ex.middle_POS
        synsets = get_synsets(s)
        for synset in synsets:
            feature_counter[synset] += 1
        
    for ex in corpus.get_examples_for_entities(kbt.obj, kbt.sbj):
        s = ex.middle_POS
        synsets = get_synsets(s)
        for synset in synsets:
            feature_counter[synset] += 1

    return feature_counter


def get_synsets(s):
    """Suggested helper method for `synset_featurizer`. This should
    be completed so that it returns a list of stringified Synsets
    associated with elements of `s`.
    """
    # Use `parse_lem` from the previous question to get a list of
    # (word, POS) pairs. Remember to convert the POS strings.
    wt = [parse_lem(lem) for lem in s.strip().split(' ') if lem]

    ##### YOUR CODE HERE
    all_synsets = []
    for w in wt:
        word = w[0]
        pos = w[1]
        normalized_pos = convert_tag(pos)
        synsets = wn.synsets(word, pos=normalized_pos)
        if len(synsets) > 0:
            for syn in synsets:
                all_synsets.append(str(syn))
                
    return all_synsets

def convert_tag(t):
    """Converts tags so that they can be used by WordNet:

    | Tag begins with | WordNet tag |
    |-----------------|-------------|
    | `N`             | `n`         |
    | `V`             | `v`         |
    | `J`             | `a`         |
    | `R`             | `r`         |
    | Otherwise       | `None`      |
    """
    if t[0].lower() in {'n', 'v', 'r'}:
        return t[0].lower()
    elif t[0].lower() == 'j':
        return 'a'
    else:
        return None


# Call to `rel_ext.experiment`:
##### YOUR CODE HERE
model_factory = lambda: LogisticRegression(fit_intercept=True, solver='liblinear')

featurizers = [synset_featurizer]

baseline_results = rel_ext.experiment(
    splits,
    train_split='train',
    test_split='dev',
    featurizers=featurizers,
    model_factory=model_factory,
    verbose=True)


relation              precision     recall    f-score    support       size
------------------    ---------  ---------  ---------  ---------  ---------
adjoins                   0.761      0.318      0.595        340       5716
author                    0.765      0.422      0.658        509       5885
capital                   0.500      0.211      0.392         95       5471
contains                  0.787      0.589      0.738       3904       9280
film_performance          0.790      0.546      0.725        766       6142
founders                  0.779      0.389      0.649        380       5756
genre                     0.500      0.194      0.380        170       5546
has_sibling               0.783      0.224      0.523        499       5875
has_spouse                0.855      0.308      0.631        594       5970
is_a                      0.602      0.225      0.451        497       5873
nationality               0.457      0.140      0.314        301       5677
parents     

In [123]:
def test_synset_featurizer(corpus):
    from collections import defaultdict
    kbt = rel_ext.KBTriple(rel='worked_at', sbj='Randall_Munroe', obj='xkcd')
    feature_counter = defaultdict(int)
    # Make sure `feature_counter` is being updated, not reinitialized:
    feature_counter["Synset('be.v.01')"] += 5
    feature_counter = synset_featurizer(kbt, corpus, feature_counter)
    # The full return values for this tend to be long, so we just
    # test a few examples to avoid cluttering up this notebook.
    test_cases = {
        "Synset('be.v.01')": 6,
        "Synset('embody.v.02')": 1
    }
    for ss, expected in test_cases.items():
        result = feature_counter[ss]
        assert result == expected, \
            "Incorrect count for {}: Expected {}; Got {}".format(ss, expected, result)

In [124]:
if 'IS_GRADESCOPE_ENV' not in os.environ:
    test_synset_featurizer(corpus)

### Your original system [3 points]

There are many options, and this could easily grow into a project. Here are a few ideas:

- Try out different classifier models, from `sklearn` and elsewhere.
- Add a feature that indicates the length of the middle.
- Augment the bag-of-words representation to include bigrams or trigrams (not just unigrams).
- Introduce features based on the entity mentions themselves. <!-- \[SPOILER: it helps a lot, maybe 4% in F-score. And combines nicely with the directional features.\] -->
- Experiment with features based on the context outside (rather than between) the two entity mentions — that is, the words before the first mention, or after the second.
- Try adding features which capture syntactic information, such as the dependency-path features used by Mintz et al. 2009. The [NLTK](https://www.nltk.org/) toolkit contains a variety of [parsing algorithms](http://www.nltk.org/api/nltk.parse.html) that may help.
- The bag-of-words representation does not permit generalization across word categories such as names of people, places, or companies. Can we do better using word embeddings such as [GloVe](https://nlp.stanford.edu/projects/glove/)?

In the cell below, please provide a brief technical description of your original system, so that the teaching team can gain an understanding of what it does. This will help us to understand your code and analyze all the submissions to identify patterns and strategies. We also ask that you report the best score your system got during development, just to help us understand how systems performed overall.

In [8]:
# PLEASE MAKE SURE TO INCLUDE THE FOLLOWING BETWEEN THE START AND STOP COMMENTS:
#   1) Textual description of your system.
#   2) The code for your original system.
#   3) The score achieved by your system in place of MY_NUMBER.
#        With no other changes to that line.
#        You should report your score as a decimal value <=1.0
# PLEASE MAKE SURE NOT TO DELETE OR EDIT THE START AND STOP COMMENTS

# IMPORT ANY MODULES BELOW THE 'IS_GRADESCOPE_ENV' CHECK CONDITION. DOING
# SO ABOVE THE CHECK MAY CAUSE THE AUTOGRADER TO FAIL.

# START COMMENT: Enter your system description in this cell.
# I used sentence embeddings from the Universal Sentence Encoder 
# to build a representation of the entire span (left, middle, and right). 
# This was then fed into the default logistic regression model for multilabel classification 
# (i.e. multiple binary classification problems). 
# My peak score was: 0.41
if 'IS_GRADESCOPE_ENV' not in os.environ:
    import tensorflow_hub as hub

    def embed(s):
        return model(s)

    def use_context_featurizer(kbt, corpus, np_func=np.mean):
        examples = corpus.get_examples_for_entities(kbt.sbj, kbt.obj)
        reverse_examples = corpus.get_examples_for_entities(kbt.obj, kbt.sbj)
        all_contexts = []

        if len(examples) > 0:
            for idx, ex in enumerate(examples):
                context = ex.left + ' ' + ex.mention_1 + ' ' + ex.middle + ' ' + ex.mention_2 + ' ' + ex.right
                # context = ex.middle
                all_contexts.append(context)

        if len(reverse_examples) > 0:
            for idx, ex in enumerate(reverse_examples):
                context = ex.left + ' ' + ex.mention_1 + ' ' + ex.middle + ' ' + ex.mention_2 + ' ' + ex.right
                all_contexts.append(context)

        # print(len(all_contexts))  
        if len(all_contexts) == 0:
            dim = 512
            return utils.randvec(n=dim)
        else:
            # print(all_contexts)
            embedding = np.array(embed(all_contexts))
            return np_func(embedding, axis=0)
        
    module_url = "https://tfhub.dev/google/universal-sentence-encoder/4" 
    model = hub.load(module_url)

    use_results = rel_ext.experiment(
        splits,
        train_split='train',
        test_split='dev',
        featurizers=[use_context_featurizer],
        vectorize=False, # Crucial for this featurizer!
        verbose=True)

# STOP COMMENT: Please do not remove this comment.

KeyboardInterrupt: 

## Bake-off [1 point]

For the bake-off, we will release a test set. The announcement will go out on the discussion forum. You will evaluate your custom model from the previous question on these new datasets using the function `rel_ext.bake_off_experiment`. Rules:

1. Only one evaluation is permitted.
1. No additional system tuning is permitted once the bake-off has started.

The cells below this one constitute your bake-off entry.

People who enter will receive the additional homework point, and people whose systems achieve the top score will receive an additional 0.5 points. We will test the top-performing systems ourselves, and only systems for which we can reproduce the reported results will win the extra 0.5 points.

Late entries will be accepted, but they cannot earn the extra 0.5 points. Similarly, you cannot win the bake-off unless your homework is submitted on time.

The announcement will include the details on where to submit your entry.

In [19]:
# Enter your bake-off assessment code in this cell.
# Please do not remove this comment.
if 'IS_GRADESCOPE_ENV' not in os.environ:
    # Please enter your code in the scope of the above conditional.
    ##### YOUR CODE HERE
    import tensorflow_hub as hub

    def embed(s):
        return model(s)

    def use_context_featurizer(kbt, corpus, np_func=np.mean):
        examples = corpus.get_examples_for_entities(kbt.sbj, kbt.obj)
        reverse_examples = corpus.get_examples_for_entities(kbt.obj, kbt.sbj)
        all_contexts = []

        if len(examples) > 0:
            for idx, ex in enumerate(examples):
                context = ex.left + ' ' + ex.mention_1 + ' ' + ex.middle + ' ' + ex.mention_2 + ' ' + ex.right
                # context = ex.middle
                all_contexts.append(context)

        if len(reverse_examples) > 0:
            for idx, ex in enumerate(reverse_examples):
                context = ex.left + ' ' + ex.mention_1 + ' ' + ex.middle + ' ' + ex.mention_2 + ' ' + ex.right
                all_contexts.append(context)

        # print(len(all_contexts))  
        if len(all_contexts) == 0:
            dim = 512
            return utils.randvec(n=dim)
        else:
            # print(all_contexts)
            embedding = np.array(embed(all_contexts))
            return np_func(embedding, axis=0)

    module_url = "https://tfhub.dev/google/universal-sentence-encoder/4" 
    model = hub.load(module_url)

    rel_ext_data_home_test = os.path.join(
        rel_ext_data_home, 'bakeoff-rel_ext-test-data'
    )
    
    featurizers = [use_context_featurizer]
    bakeoff_results = rel_ext.experiment(
        splits,
        train_split='train',
        test_split='dev',
        featurizers=featurizers,
        verbose=False,
        vectorize=False,
    ) # We don't care about this eval, so skip its summary.
    
    rel_ext.bake_off_experiment(bakeoff_results, rel_ext_data_home_test)

relation              precision     recall    f-score    support       size
------------------    ---------  ---------  ---------  ---------  ---------
adjoins                   0.481      0.176      0.357        438       7122
author                    0.662      0.411      0.590        645       7329
capital                   0.667      0.017      0.079        115       6799
contains                  0.711      0.759      0.720       3808      10492
film_performance          0.670      0.581      0.650       1011       7695
founders                  0.583      0.198      0.420        444       7128
genre                     0.500      0.059      0.199        188       6872
has_sibling               0.804      0.319      0.617        717       7401
has_spouse                0.740      0.346      0.603        780       7464
is_a                      0.650      0.219      0.467        611       7295
nationality               0.805      0.162      0.449        383       7067
parents     

In [ ]:
# On an otherwise blank line in this cell, please enter
# your macro-average f-score (an F_0.5 score) as reported
# by the code above. Please enter only a number between
# 0 and 1 inclusive. Please do not remove this comment.
if 'IS_GRADESCOPE_ENV' not in os.environ:
    # Please enter your score in the scope of the above conditional.
    ##### YOUR CODE HERE
    0.424